# Converting the raw data ready to pocess, to csv file


### all the imports

In [110]:
import os
import pandas as pd


### getting all the files from the directory

In [101]:
def get_files():
    all_file_path = []
    """walk through the data dir to its sub-dirs and gets all the file with .review extension and return all the file_paths list"""
    # looping through the directory and the sub-directory
    for root, dirs, files in os.walk("data"):
        for file in files:
            # getting the file end with .review
            if file.endswith(".review"):
                # joining the file path with the actual full path of the file 
                file_path = os.path.join(root, file)
                # appending the file_path in the list 
                all_file_path.append(file_path)
    return all_file_path


## categorized the files into list


In [102]:
def categorise_files(all_file_path):
    """get all file path and categorised the files into positive, negative and unlabeled review and return the file path list as tuple"""
    
    pos_review_f_path = []
    neg_review_f_path=[]
    unl_review_f_path = []
    
    for file in all_file_path:
        if "positive" in file:
            pos_review_f_path.append(file)
        elif "negative" in file:
            neg_review_f_path.append(file)
        else:
            unl_review_f_path.append(file)
    return pos_review_f_path, neg_review_f_path, unl_review_f_path

## reading the files

In [84]:
def read_file(file_path):
    """reads the file from file path and return all lines in list"""
    with open(file_path) as file:
        items = file.readlines()
    return items

## making the list of review after reading the file

In [103]:
def build_review_list(items):
    """gets all the items, prepare a list of each review and returns the list of reviews"""
    reviews = []
    review = []
    for item in items:
        # remove \n from the item
        item = item[:-1]
        # get the file inside the review block not including the start and the end block
        if item != '<review>' and item != '</review':
            review.append(item)
            
        #  adding the review to the review list  
        if item == '</review>':
            reviews.append(review)
            review = []
    return reviews
   

### processing the review and making it ready to transfer into the dataframe

In [104]:
def process_review_list(list_of_reviews):
    """gets the list of reviews, seperate the headings and values and returns as list of dictionary"""
    dict_review_list = []
    previous_key = None
    
    #iterating through reviews
    for review in (list_of_reviews):
        dict_review = {}
        #iterating through a single review
        for num, item in enumerate(review):
            #eliminating null values
            if item != "":
                #getting and heading and assigning with corresponding values
                if item[0] =="<" and item[1] != "/":
                    dict_review[item] = review[num + 1]
                    previous_key = item
                #checking if any values are missing and reassigning them
                if item not in dict_review.values() and item[0] != '<':
                    dict_review[previous_key] += item
        # when a review is ready adding it to the list
        dict_review_list.append(dict_review)
    return dict_review_list

### converting the list of the dict into the datarame to convert into the csv file or process straightly


In [105]:
# creating the data from list of dict using lambda
to_dataframe = lambda list_of_dict: pd.DataFrame(list_of_dict)

### converting the dataframe into the csv file and saving into the directory


In [106]:
# convert and save to csv file
save_to_csv = lambda df, file_name: df.to_csv(file_name, index=False, encoding='utf-8')

### combining all the process to get single list of review of each category file found in directory

In [107]:

def convert_to_csv():
    dir = os.path.abspath(os.getcwd()) + "/data/csv_files/"
    
    file_name = ('positive_review', 'negative_review', 'unlabled_review')

    # get all the files
    all_file_path = get_files()
    
    #categorised the files in the list
    categories = categorise_files(all_file_path)
    
    #read all the files from each category:
    for num, category in enumerate(categories):
        
        # hold all the items of each category in one list 
        items_holder = []
        
        for file_path in category:
            # getting all the items
            items = read_file(file_path)
            items_holder.extend(items)
        
        # build the list of the items in one category
        list_of_reviews = build_review_list(items_holder)
        
        # buid of list of processed review in dictionaries
        list_of_dict_reviews = process_review_list(list_of_reviews)
        
        # build of dataframe from processed_list
        data_frame = to_dataframe(list_of_dict_reviews)
        
        # save the dataframe into the csv file
        save_to_csv(data_frame, file_name=(dir + file_name[num]))
        
            

In [111]:
# convert to csv
convert_to_csv()

# Processing the data 

In [175]:
# options for pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_colwidth')

In [136]:
# opening  csv file
open_review = lambda file: pd.read_csv(file)

In [137]:
df_positive = open_review("./data/csv_files/positive_review")
df_negative = open_review("./data/csv_files/negative_review")

In [ ]:
# remove the unique id and make asin as a unique id
# remove the unwanted text from product name
# in helpful column make the data in fractions or ratio to represent mathmatically
# convert the date column into the date type
# remove the unwanted text from the review text

In [ ]:
# process to follow

# process the positive review file
# process the negative review file
# merge the both file 
# randomised the data
# split the data into train, evaluation and test data
# train the model 
# evaluate the model
# test the model
# dump the model into the file so that we don't have to train the new model time to time


In [176]:
# we can replace the nan reviewer to "Anonymous User" to say these person does not want to disclose their names on reviews
# it can be possible that the reviewer doesn't have amazon account, or he posted anonymoulsly or there is an technical issues in the database
df_positive.loc[df_positive['<reviewer>'].isna()].head()

,<unique_id>,<asin>,<product_name>,<product_type>,<helpful>,<rating>,<title>,<date>,<reviewer>,<reviewer_location>,<review_text>
238,"B000EP5MOA:excellent_ipod,_my_favorite_so_far.:",B000EP5MOA,Apple 4 GB iPod Nano Green (2nd Generation),electronics,7 of 7,5.0,"Excellent iPod, my favorite so far.","November 22, 2006",NaN,NaN,I love the new nanos. I don't think I have any...
316,B00001P4ZD:fits_my_fat_head...:,B00001P4ZD,Koss KTX-Pro Portable Stereophone,electronics,1 of 2,4.0,Fits my fat head...,"August 21, 2002",NaN,NaN,I stretched them till they fit comfortably on ...
433,B00000J0O2:it_does_the_job:,B00000J0O2,HP Microfine Toner Cartridge for LaserJet 5L/6...,electronics,3 of 3,5.0,It does the job,"June 20, 2000",NaN,NaN,Well the most important thing about buying ink...
438,"B000058E2B:why_i_still_like_this_pda,_continued:",B000058E2B,Compaq iPAQ 3635 Pocket PC Bundle (with Compac...,electronics,NaN,4.0,"Why I still like this PDA, continued","April 4, 2004",NaN,NaN,I found out the way to copy photos from my Com...
532,B00003CWE5:buy_these_cables!:,B00003CWE5,Monster Cable MV2CV-1M Monster Video 2 Compone...,electronics,4 of 12,5.0,Buy these cables!,"March 15, 2001",NaN,NaN,These cables are super with my new DVD player....


In [ ]:
# looks like all the nan reviewer also has nan values in the location table,
# which can also provides more clarity that, the order without creating the amazon accounts
# or they have incomplete account setup 

In [140]:
# truncate the ':electronics' from the product name column values
df_positive['<product_name>'] = df_positive['<product_name>'].str.partition(":")[0]

In [156]:
#working with the unique id and the reviewer
df_positive['<unique_id>'][98]

'B00083LFSU:great:malachite'

In [122]:
df_negative.isnull().sum()

<unique_id>              0
<asin>                   0
<product_name>           0
<product_type>           0
<helpful>              467
<rating>                 0
<title>                  0
<date>                   1
<reviewer>              81
<reviewer_location>    723
<review_text>            0
dtype: int64

# Making the Model Ready